In [1]:
# Auto-Reload Setup für Entwicklung
# Automatisches Neuladen aller Module bei Änderungen
%load_ext autoreload
%autoreload 2

In [2]:
# Imports - werden automatisch neu geladen bei Änderungen
from services.parsers.parse_vitals_data import parse_vitals_data
from services.parsers.parse_respiratory_data import parse_respiratory_data
from services.parsers.parse_lab_data import parse_lab_data
from services.parsers.parse_ecmo_data import parse_ecmo_data
from services.parsers.parse_impella_data import parse_impella_data
from services.parsers.parse_crrt_data import parse_crrt_data
from services.parsers.parse_medication_data import parse_medication_data
from services.parsers.parse_fluidbalance_data import parse_fluidbalance_data
from state_provider.state_provider_class import state_provider
from datetime import datetime

print("✅ Module erfolgreich importiert mit Auto-Reload aktiviert")

✅ Module erfolgreich importiert mit Auto-Reload aktiviert


In [3]:
# Daten laden und parsen
CSV = "data/gesamte_akte2.csv"
file = open(CSV, "r", encoding="utf-8").read()
DELIMITER = "|"
state = state_provider.parse_data_to_state(file, DELIMITER)

print("✅ Daten erfolgreich geparst")

2025-10-31 16:13:09.847 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-31 16:13:09.848 WARNING streamlit.runtime.state.session_state_proxy: Session state does not function when running a script without `streamlit run`
2025-10-31 16:13:09.849 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-31 16:13:09.849 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-31 16:13:09.849 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-31 16:13:18.537 WARNING streamlit.runtime.scriptrunner_utils.script_run_c

✅ Daten erfolgreich geparst


In [4]:
# Hilfsfunktion: Alle verfügbaren Methoden anzeigen
def show_available_methods():
    methods = [method for method in dir(state_provider) if not method.startswith('_')]
    print("Verfügbare StateProvider Methoden:")
    for i, method in enumerate(methods, 1):
        print(f"  {i:2}. {method}")
    return methods

# Methoden anzeigen
available_methods = show_available_methods()

Verfügbare StateProvider Methoden:
   1. data_parser
   2. get_device_time_ranges
   3. get_respiration_type
   4. get_respiratory_value
   5. get_selected_view
   6. get_state
   7. get_time_of_mcs
   8. get_time_range
   9. get_vasoactive_agents_df
  10. get_vitals_value
  11. has_device_past_24h
  12. has_mcs_records_past_24h
  13. has_parsed_data
  14. parse_data_to_state
  15. query_data
  16. reset_state
  17. save_state
  18. set_selected_time_range
  19. update_state


In [5]:
# get's last date of ecmo as example of a day with full mcs support
from services.date_as_datetime import get_date_as_datetime
impella = state_provider.get_device_time_ranges("hämofilter  ")
print(impella)

2025-10-31 16:13:18.579 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-31 16:13:18.579 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


                                     category       start         end
0  Hämofilter   V. jugularis interna links  1  2025-09-14  2025-09-15
1  Hämofilter   V. jugularis interna links  2  2025-09-16  2025-09-18
2  Hämofilter   V. jugularis interna links  3  2025-09-18  2025-09-24
3  Hämofilter   V. jugularis interna links  4  2025-09-24  2025-09-25


In [6]:
from services.data_parser import DataParser
parser = DataParser(file, ";")

In [7]:
vitals = state_provider.query_data("vitals")
vitals

2025-10-31 16:13:18.637 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-31 16:13:18.638 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


,timestamp,category,parameter,value
0,2025-09-11 18:53:00,Online erfasste Vitaldaten,HF [min⁻¹],99.0
1,2025-09-11 19:53:00,Online erfasste Vitaldaten,HF [min⁻¹],98.0
2,2025-09-11 20:53:00,Online erfasste Vitaldaten,HF [min⁻¹],94.0
3,2025-09-11 21:53:00,Online erfasste Vitaldaten,HF [min⁻¹],92.0
4,2025-09-11 22:53:00,Online erfasste Vitaldaten,HF [min⁻¹],89.0
...,...,...,...,...
3373,2025-09-18 21:53:00,Manuell erfasste Vitaldaten,HZV (l/min),1.3
3374,2025-09-19 06:53:00,Manuell erfasste Vitaldaten,HZV (l/min),594.0
3375,2025-09-19 08:53:00,Manuell erfasste Vitaldaten,HZV (l/min),1.7
3376,2025-09-19 13:53:00,Manuell erfasste Vitaldaten,HZV (l/min),2.2


In [8]:
# Filter rows where the category contains "Online"
# use .str.contains to get a boolean mask (safer than "in" on a Series)
test_date = datetime.strptime("2025-09-16", "%Y-%m-%d").date()
test_date2 = datetime(2025,9,14).date() #geht auch so - für die Zukunft ;)
art = state_provider.query_data("vitals", {'timestamp': test_date, 'parameter': "ARTs", 'value_strategy': "median"})
art

2025-10-31 16:13:18.658 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-31 16:13:18.658 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


,timestamp,category,parameter,value


In [9]:
supra = state_provider.query_data("medication", {"medication": "supra", "start": test_date})
heparin = state_provider.query_data("medication", {"medication": "heparin", "stop": test_date2})
heparin

2025-10-31 16:13:18.677 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-31 16:13:18.677 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-31 16:13:18.680 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-31 16:13:18.680 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


,medication,category,application,start,stop,concentration,rate
0,Heparin Perfusor Heparin-Natrium-5000-ratiop...,Perfusoren,Perfusor,2025-09-13 13:30:00,2025-09-14 02:24:00,"25000 I.E. 1 mL 5 Amp. 7,546 mmol 49 mL 0,98 ...",1.0
1,Heparin Perfusor Heparin-Natrium-5000-ratiop...,Perfusoren,Perfusor,2025-09-14 02:24:00,2025-09-14 02:38:00,"25000 I.E. 1 mL 5 Amp. 7,546 mmol 49 mL 0,98 ...",0.8
2,Heparin Perfusor Heparin-Natrium-5000-ratiop...,Perfusoren,Perfusor,2025-09-14 02:38:00,2025-09-14 15:04:00,"25000 I.E. 1 mL 5 Amp. 7,546 mmol 49 mL 0,98 ...",0.5
3,Heparin Perfusor Heparin-Natrium-5000-ratiop...,Perfusoren,Perfusor,2025-09-14 15:04:00,2025-09-14 22:28:00,"25000 I.E. 1 mL 5 Amp. 7,546 mmol 49 mL 0,98 ...",0.5


In [10]:
from datetime import time
near_lactat = state_provider.query_data("lab", {"timestamp": datetime(2025,9,15).date(), "parameter": "Lactat","category": "Blutgase arteriell", "value_strategy": {"nearest": time(12, 0)}})
near_lactat

# geht!

2025-10-31 16:13:18.746 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-31 16:13:18.746 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
Unknown value_strategy '{'nearest': datetime.time(12, 0)}' requested for lab


,timestamp,category,parameter,value
0,2025-09-15 00:53:00,Blutgase arteriell,LACTAT(BG) [mg·dL⁻¹],31.0
1,2025-09-15 02:23:00,Blutgase arteriell,LACTAT(BG) [mg·dL⁻¹],31.0
2,2025-09-15 03:53:00,Blutgase arteriell,LACTAT(BG) [mg·dL⁻¹],26.0
3,2025-09-15 04:23:00,Blutgase arteriell,LACTAT(BG) [mg·dL⁻¹],24.0
4,2025-09-15 05:23:00,Blutgase arteriell,LACTAT(BG) [mg·dL⁻¹],24.0
5,2025-09-15 06:53:00,Blutgase arteriell,LACTAT(BG) [mg·dL⁻¹],19.0
6,2025-09-15 08:53:00,Blutgase arteriell,LACTAT(BG) [mg·dL⁻¹],16.0
7,2025-09-15 11:23:00,Blutgase arteriell,LACTAT(BG) [mg·dL⁻¹],16.0
8,2025-09-15 14:23:00,Blutgase arteriell,LACTAT(BG) [mg·dL⁻¹],14.0
9,2025-09-15 16:23:00,Blutgase arteriell,LACTAT(BG) [mg·dL⁻¹],16.0


In [11]:
state_provider.query_data("impella")

2025-10-31 16:13:18.784 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-31 16:13:18.784 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


,timestamp,value,category,parameter,source_header
0,2025-09-15 20:55:00,blutig,Impella A. axilliaris rechts 1,Beobachtung,Impella A. axilliaris rechts
1,2025-09-15 20:55:00,3.4,Impella A. axilliaris rechts 1,HZV (l/min),Impella A. axilliaris rechts
2,2025-09-15 20:55:00,ultraschallgestützt (TEE),Impella A. axilliaris rechts 1,HZV (l/min),Impella A. axilliaris rechts
3,2025-09-15 20:55:00,Impella implantiert,Impella A. axilliaris rechts 1,Zustand,Impella A. axilliaris rechts
4,2025-09-16 01:21:00,1.4,Impella A. axilliaris rechts 1,HZV (l/min),Impella A. axilliaris rechts
...,...,...,...,...,...
84,2025-09-19 14:29:00,Verband trocken,Impella A. axilliaris rechts 1,Beobachtung,Impella A. axilliaris rechts
85,2025-09-19 14:29:00,2.2,Impella A. axilliaris rechts 1,HZV (l/min),Impella A. axilliaris rechts
86,2025-09-19 14:29:00,513.0,Impella A. axilliaris rechts 1,Purgedruck in mmHg,Impella A. axilliaris rechts
87,2025-09-19 14:29:00,6.6,Impella A. axilliaris rechts 1,Purgefluß in ml/h,Impella A. axilliaris rechts


In [ ]:
from services.value_aggregation.lab_aggregator import LabAggregator
from datetime import date
lab_aggregator = LabAggregator(state_provider, date= date(2025, 9, 19), record_id=state_provider.get_record_id(), redcap_event_name="impella_arm_2", redcap_repeat_instrument="labor", redcap_repeat_instance=1, value_strategy="nearest", nearest_time=state_provider.get_impella_implant_time())


2025-10-31 16:34:40.776 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-31 16:34:40.778 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


In [33]:
entry = lab_aggregator.create_lab_entry()

2025-10-31 16:34:43.093 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-31 16:34:43.094 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
nearest_time required for value_strategy='nearest', got <class 'NoneType'>
2025-10-31 16:34:43.169 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-31 16:34:43.169 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
nearest_time required for value_strategy='nearest', got <class 'NoneType'>
2025-10-31 16:34:43.176 WARNING streamlit.runtime.scriptrunner_utils.script_run_contex

In [34]:
entry

LabModel(record_id='123wqewer', redcap_event_name='impella_arm_2', redcap_repeat_instrument='labor', redcap_repeat_instance=1.0, na_post_2=1.0, ecmella_2=0.0, assess_time_point_labor=1.0, assess_date_labor=datetime.date(2025, 9, 19), art_site=7, pc02=44.8, p02=35.0, ph=7.376, hco3=25.6, be=0.8, sa02=61.9, k=4.1, na=136.0, gluc=101.0, lactate=10.0, sv02=61.9, wbc=7.8, hb=12.5, hct=29.1, plt=73.0, ptt=70.7, quick=51.0, inr=1.6, post_act=None, act=None, ck=4192.0, ckmb=82.0, got=None, alat=None, ggt=None, ldh=None, lipase=None, albumin=None, post_crp=None, crp=None, post_pct=None, pct=15.1, hemolysis=None, fhb=12.5, haptoglobin=None, bili=3.2, crea=2.2, cc=31.18, urea=36.1, labor_complete=None)

In [35]:
# Test: Enum-Werte als Integer exportieren
entry.model_dump(mode='json')

{'record_id': '123wqewer',
 'redcap_event_name': 'impella_arm_2',
 'redcap_repeat_instrument': 'labor',
 'redcap_repeat_instance': 1.0,
 'na_post_2': 1.0,
 'ecmella_2': 0.0,
 'assess_time_point_labor': 1.0,
 'assess_date_labor': '2025-09-19',
 'art_site': 7,
 'pc02': 44.8,
 'p02': 35.0,
 'ph': 7.376,
 'hco3': 25.6,
 'be': 0.8,
 'sa02': 61.9,
 'k': 4.1,
 'na': 136.0,
 'gluc': 101.0,
 'lactate': 10.0,
 'sv02': 61.9,
 'wbc': 7.8,
 'hb': 12.5,
 'hct': 29.1,
 'plt': 73.0,
 'ptt': 70.7,
 'quick': 51.0,
 'inr': 1.6,
 'post_act': None,
 'act': None,
 'ck': 4192.0,
 'ckmb': 82.0,
 'got': None,
 'alat': None,
 'ggt': None,
 'ldh': None,
 'lipase': None,
 'albumin': None,
 'post_crp': None,
 'crp': None,
 'post_pct': None,
 'pct': 15.1,
 'hemolysis': None,
 'fhb': 12.5,
 'haptoglobin': None,
 'bili': 3.2,
 'crea': 2.2,
 'cc': 31.18,
 'urea': 36.1,
 'labor_complete': None}